In [1]:
#topic modelling on original english tweets dataset of size 16575 to generate 10 distinct topics

In [2]:
#importing libraraies
import pandas as pd
import os, re, csv
from csv import writer
from csv import reader
import gensim
from gensim.utils import simple_preprocess
from operator import itemgetter 

In [3]:
processeddata = pd.read_csv('tweets[processed_english].csv')

In [4]:
def tokenize(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
            result.append(token)
    return result

In [5]:
#dataset labeling

In [6]:
#map is used to tokenize
processed_corpus = processeddata['text'].map(tokenize)

In [7]:
dictionary = gensim.corpora.Dictionary(processed_corpus)

In [8]:
print dictionary

Dictionary(26494 unique tokens: [u'minkasadhupyog', u'sivashiv', u'sation', u'disobeying', u'carryminati']...)


In [9]:
dictionary.filter_extremes(no_below=30, keep_n=1000)

In [10]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [12]:
for i, words in lda_model.print_topics():
    print('topic:{} \nlist_of_words: {}'.format( i, words))
    print('\n')

topic:0 
list_of_words: 0.438*"aadharcard" + 0.165*"number" + 0.110*"aadhaar" + 0.070*"mobile" + 0.057*"details" + 0.026*"modi" + 0.021*"form" + 0.018*"says" + 0.017*"application" + 0.016*"birth"


topic:1 
list_of_words: 0.216*"india" + 0.181*"narendramodi" + 0.095*"pmoindia" + 0.076*"take" + 0.074*"office" + 0.036*"st" + 0.035*"work" + 0.030*"making" + 0.029*"used" + 0.026*"action"


topic:2 
list_of_words: 0.144*"address" + 0.109*"new" + 0.102*"proof" + 0.066*"photo" + 0.065*"give" + 0.059*"airtel" + 0.056*"asked" + 0.046*"information" + 0.044*"centre" + 0.031*"wrong"


topic:3 
list_of_words: 0.096*"video" + 0.084*"made" + 0.065*"good" + 0.064*"said" + 0.063*"plz" + 0.049*"news" + 0.046*"delhi" + 0.044*"country" + 0.041*"much" + 0.041*"thank"


topic:4 
list_of_words: 0.085*"linked" + 0.079*"kyc" + 0.067*"got" + 0.064*"paytm" + 0.051*"totally" + 0.049*"getting" + 0.039*"days" + 0.038*"again" + 0.034*"ask" + 0.033*"service"


topic:5 
list_of_words: 0.122*"need" + 0.120*"pan" + 0.06

In [13]:
#analysing new sentence(tweet) by % of belonging from each topic
lda_model[dictionary.doc2bow(tokenize("mobile sim number linked with adhaar"))][0]

[(0, 0.13312648),
 (1, 0.03956868),
 (2, 0.040368736),
 (3, 0.032686394),
 (4, 0.086468145),
 (5, 0.06586392),
 (6, 0.054633163),
 (7, 0.018266905),
 (8, 0.05149425),
 (9, 0.07442124),
 (10, 0.053086374),
 (11, 0.024997832),
 (12, 0.0351393),
 (13, 0.046823844),
 (14, 0.02276962),
 (15, 0.055411257),
 (16, 0.02569849),
 (17, 0.010889462),
 (18, 0.08371193),
 (19, 0.044573966)]

In [14]:
#dataset annotation based on generated 10 topics

In [15]:
#loading and reading dataset
#tweets[processed_english] has 16575 tweets
#these tweets are detected as english using BLD model
english_tweets = open("tweets[processed_english].csv", "r")
english_tweets = english_tweets.read()

In [16]:
#cleaning of tweets
#elimination of word whose length is less than 3
#elimination of tweets which has word length less than 5 and more than 15
#creation of new csv file tweets[more_processed_english].csv
for line in english_tweets.split('\n'):
    if line:
        sentence=[]
        for word in line.split(' '):
            if len(word)>2:
                sentence.append(word)
        if len(sentence)>=5 and len(sentence)<=15:
            with open('tweets[processed_english_for_labeling].csv', 'a') as csvFile:
                writer = csv.writer(csvFile)
                writer.writerow(sentence)
            csvFile.close()
with open('tweets[processed_english_for_labeling].csv') as f:
    f=f.read()
    re1 = re.compile(',')
    file = re1.sub(' ', f)
with open('tweets[processed_english_for_labeling].csv','w') as F:
    F.write(file)
    F.close()


In [17]:
#visualizing and labelling topic no to a new tweet based on max % of belonging from that topic
test_tweet="mobile sim number linked with adhaar"
lda_model[dictionary.doc2bow(tokenize(test_tweet))][0]
#clearly 6 is topic no as more than 20 % of belonging is from that topic
topic_no=max(lda_model[dictionary.doc2bow(tokenize(test_tweet))][0], key = itemgetter(1))[0]
value=max(lda_model[dictionary.doc2bow(tokenize(test_tweet))][0], key = itemgetter(1))[1]
print "topic no:", topic_no," value:", value

topic no: 0  value: 0.1331265


In [18]:
# Opening tweets[more_processed_english].csv in read mode and semi_labeled.csv in write mode
# Only those tweets which has atleast 20% of belonging from the topics and topic no corresponding to maximum value of belonging will be assigned a label
from csv import writer
from csv import reader
with open('tweets[processed_english_for_labeling].csv', 'r') as read_obj, \
        open('tweets[labeled].csv', 'w') as write_obj:
    csv_reader = reader(read_obj)
    csv_writer = writer(write_obj)
    for row in csv_reader:
        #if (max(lda_model[dictionary.doc2bow(tokenize(row[0]))][0], key = itemgetter(1))[1])>.2:
        row.append(max(lda_model[dictionary.doc2bow(tokenize(row[0]))][0], key = itemgetter(1))[0])
        csv_writer.writerow(row)